# **Prueba 1:**

In [38]:
from datetime import datetime

class Producto:
    def __init__(self, nombre, fecha_vencimiento=None, perecible=False):
        self.nombre = nombre
        # Convertir la fecha de vencimiento a un objeto datetime si se proporciona
        self.fecha_vencimiento = datetime.strptime(fecha_vencimiento, "%Y-%m-%d") if fecha_vencimiento else None
        self.perecible = perecible

class Nodo:
    def __init__(self, tipo):
        self.tipo = tipo  # Tipo de producto (cárnicos, lácteos, conservas, etc.)
        self.productos = []  # Lista de productos de este tipo
        self.hijos = []  # Subcategorías o más productos

    def agregar_producto(self, producto):
        self.productos.append(producto)
        if producto.perecible:
            # Ordenar productos perecibles por fecha de vencimiento, más próximo primero
            self.productos.sort(key=lambda x: x.fecha_vencimiento)

    def agregar_hijo(self, nodo):
        self.hijos.append(nodo)

    def buscar_producto_por_tipo(self, tipo):
        # Búsqueda por tipo de producto en el árbol
        if self.tipo == tipo:
            return self.productos
        for hijo in self.hijos:
            resultado = hijo.buscar_producto_por_tipo(tipo)
            if resultado:
                return resultado
        return []

    def buscar_productos_perecibles(self):
        """Devuelve una lista de productos perecibles en este nodo y sus hijos."""
        productos_perecibles = []
        
        # Agregar productos perecibles del nodo actual
        for producto in self.productos:
            if producto.perecible:
                productos_perecibles.append(producto)
        
        # Agregar productos perecibles de los hijos
        for hijo in self.hijos:
            productos_perecibles.extend(hijo.buscar_productos_perecibles())
            
        return productos_perecibles
    
    def buscar_producto_por_vencimiento(self, fecha):
        # Convertir la fecha de búsqueda a un objeto datetime
        fecha_busqueda = datetime.strptime(fecha, "%Y-%m-%d")
        productos_vencidos = []
        for producto in self.productos:
            if producto.perecible and producto.fecha_vencimiento <= fecha_busqueda:
                productos_vencidos.append(producto)
        for hijo in self.hijos:
            productos_vencidos.extend(hijo.buscar_producto_por_vencimiento(fecha))
        return productos_vencidos


# Crear un árbol genérico de productos
arbol_productos = Nodo("Almacén")

# Crear nodos para tipos de productos
frutas = Nodo("Frutas")
verduras = Nodo("Verduras")
conservas = Nodo("Conservas")
granos = Nodo("Granos")

# Agregar los nodos de tipos de productos al árbol principal
arbol_productos.agregar_hijo(frutas)
arbol_productos.agregar_hijo(verduras)
arbol_productos.agregar_hijo(conservas)
arbol_productos.agregar_hijo(granos)
 
# Agregar productos a cada tipo
frutas.agregar_producto(Producto("Manzana", "2024-11-10", perecible=True))
frutas.agregar_producto(Producto("Melocoton", "2024-11-15", perecible=True))
frutas.agregar_producto(Producto("Platano", "2024-12-01", perecible=True))
verduras.agregar_producto(Producto("Lechuga", "2024-10-30", perecible=True))
verduras.agregar_producto(Producto("Rabanito", "2024-11-10", perecible=True))
verduras.agregar_producto(Producto("Brocoli", "2024-11-05", perecible=True))
conservas.agregar_producto(Producto("Filete de atún en aceite vegetal", perecible=False))
conservas.agregar_producto(Producto("Conserva de piña en almíbar", perecible=False))
conservas.agregar_producto(Producto("Conserva de durazno", perecible=False))
granos.agregar_producto(Producto("Avena", perecible=False))
granos.agregar_producto(Producto("Arroz", perecible=False))
granos.agregar_producto(Producto("Quinoa", perecible=False))
granos.agregar_producto(Producto("Pasta", perecible=False))

In [39]:
class NodoProducto:
    def __init__(self, producto):
        self.producto = producto  # Almacena el producto
        self.siguiente = None  # Puntero al siguiente nodo

In [60]:
class PilaProductosPerecibles:
    def __init__(self):
        self.cima = None

    def esta_vacia(self):
        return self.cima is None

    def push(self, producto):
        if not producto.perecible:
            raise ValueError("Solo se pueden agregar productos perecibles a la pila.")
        
        nuevo_nodo = NodoProducto(producto)
        nuevo_nodo.siguiente = self.cima
        self.cima = nuevo_nodo

    def pop(self):
        if self.esta_vacia():
            raise IndexError("La pila está vacía")
        producto = self.cima.producto
        self.cima = self.cima.siguiente
        return producto

    def peek(self):
        if self.esta_vacia():
            raise IndexError("La pila está vacía")
        return self.cima.producto

    def mostrar_pila(self):
        nodos = []
        actual = self.cima
        while actual:
            nodos.append(actual.producto.nombre)
            actual = actual.siguiente
        print(" -> ".join(nodos))

    def llenar_desde_arbol(self, arbol):
        """Llena la pila con productos perecibles desde el árbol."""
        productos_perecibles = arbol.buscar_productos_perecibles()
        for producto in productos_perecibles:
            self.push(producto)
    
    def filtrar_por_fecha(self, fecha_ingresada):
        fecha_busqueda = datetime.strptime(fecha_ingresada, "%Y-%m-%d")
        actual = self.cima
        print(f"\nProductos que vencen después de {fecha_ingresada}:")

        while actual:
            if actual.producto.fecha_vencimiento > fecha_busqueda:
                print(f"Producto: {actual.producto.nombre}, Fecha de vencimiento: {actual.producto.fecha_vencimiento.strftime('%Y-%m-%d')}")
            actual = actual.siguiente
    
    def mostrar_menor_fecha(self, fecha_ingresada):
        fecha_busqueda = datetime.strptime(fecha_ingresada, "%Y-%m-%d")
        actual = self.cima
        print(f"\nProductos que vencen antes de {fecha_ingresada}:")

        while actual:
            if actual.producto.fecha_vencimiento < fecha_busqueda:
                print(f"Producto: {actual.producto.nombre}, Fecha de vencimiento: {actual.producto.fecha_vencimiento.strftime('%Y-%m-%d')}")
            actual = actual.siguiente

In [61]:
pila_perecibles = PilaProductosPerecibles()
pila_perecibles.llenar_desde_arbol(arbol_productos)

In [65]:
print("Productos perecibles en la pila:")
pila_perecibles.mostrar_pila()

Productos perecibles en la pila:
Rabanito -> Brocoli -> Lechuga -> Platano -> Melocoton -> Manzana


In [66]:
fecha_ingresada = input("Ingrese la fecha de vencimiento que desea ver: ")
pila_perecibles.filtrar_por_fecha(fecha_ingresada)


Productos que vencen después de 2024-11-15:
Producto: Platano, Fecha de vencimiento: 2024-12-01


In [67]:
fecha_ingresada = input("Ingrese la fecha de vencimiento que desea ver: ")
pila_perecibles.mostrar_menor_fecha(fecha_ingresada)


Productos que vencen antes de 2024-11-15:
Producto: Rabanito, Fecha de vencimiento: 2024-11-10
Producto: Brocoli, Fecha de vencimiento: 2024-11-05
Producto: Lechuga, Fecha de vencimiento: 2024-10-30
Producto: Manzana, Fecha de vencimiento: 2024-11-10


# **Prueba 2:**

In [1]:
import json
from datetime import datetime
from collections import defaultdict

# Clase Producto
class Producto:
    def __init__(self, nombre, fecha_vencimiento=None, perecible=False, cantidad=0, categoria=None):
        self.nombre = nombre
        self.fecha_vencimiento = datetime.strptime(fecha_vencimiento, "%Y-%m-%d") if fecha_vencimiento else None
        self.perecible = perecible
        self.cantidad = int(cantidad)  # Convertir a entero
        self.categoria = categoria
        self.siguiente = None  # Enlace al siguiente producto

# Clase Nodo
class Nodo:
    def __init__(self, tipo):
        self.tipo = tipo
        self.producto = None
        self.hijo = None  # Primer subcategoría o producto
        self.hermano = None  # Siguiente nodo del mismo nivel

    def agregar_producto(self, nuevo_producto):
        if not self.producto:
            self.producto = nuevo_producto
        else:
            actual = self.producto
            anterior = None
            while actual and (not nuevo_producto.perecible or 
                              (nuevo_producto.perecible and nuevo_producto.fecha_vencimiento >= actual.fecha_vencimiento)):
                anterior = actual
                actual = actual.siguiente
            nuevo_producto.siguiente = actual
            if anterior:
                anterior.siguiente = nuevo_producto
            else:
                self.producto = nuevo_producto

    def agregar_hijo(self, nodo_hijo):
        if not self.hijo:
            self.hijo = nodo_hijo
        else:
            actual = self.hijo
            while actual.hermano:
                actual = actual.hermano
            actual.hermano = nodo_hijo

    def buscar_producto_por_tipo(self, tipo):
        if self.tipo == tipo:
            return self
        actual = self.hijo
        while actual:
            resultado = actual.buscar_producto_por_tipo(tipo)
            if resultado:
                return resultado
            actual = actual.hermano
        return None
    
    def buscar_producto_por_vencimiento(self, fecha):
        fecha_busqueda = datetime.strptime(fecha, "%Y-%m-%d")
        producto_actual = self.producto
        while producto_actual:
            if producto_actual.perecible and producto_actual.fecha_vencimiento <= fecha_busqueda:
                print(f"Producto que vence antes del {fecha}: {producto_actual.nombre}")
            producto_actual = producto_actual.siguiente
        hijo = self.hijo
        while hijo:
            hijo.buscar_producto_por_vencimiento(fecha)
            hijo = hijo.hermano

# Clase Arbol
class Arbol:
    def __init__(self, tipo_raiz):
        self.raiz = Nodo(tipo_raiz)

    def agregar_tipo(self, nodo_tipo):
        self.raiz.agregar_hijo(nodo_tipo)

    def agregar_producto_a_tipo(self, tipo, producto):
        nodo_tipo = self.raiz.buscar_producto_por_tipo(tipo)
        if nodo_tipo:
            nodo_tipo.agregar_producto(producto)
        else:
            print(f"Tipo de producto '{tipo}' no encontrado.")
    
    def buscar_por_tipo(self, tipo):
        nodo_tipo = self.raiz.buscar_producto_por_tipo(tipo)
        return nodo_tipo.producto if nodo_tipo else None

    def buscar_por_vencimiento(self, fecha):
        self.raiz.buscar_producto_por_vencimiento(fecha)

    def calcular_suma_cantidad_por_tipo(self):
        cantidades = defaultdict(int)
        self._calcular_suma_cantidad_por_tipo_recursivo(self.raiz, cantidades)
        return dict(cantidades)

    def _calcular_suma_cantidad_por_tipo_recursivo(self, nodo, cantidades):
        if nodo.producto:
            producto_actual = nodo.producto
            while producto_actual:
                cantidades[nodo.tipo] += producto_actual.cantidad
                producto_actual = producto_actual.siguiente
        if nodo.hijo:
            self._calcular_suma_cantidad_por_tipo_recursivo(nodo.hijo, cantidades)
        if nodo.hermano:
            self._calcular_suma_cantidad_por_tipo_recursivo(nodo.hermano, cantidades)

    def calcular_suma_cantidad_por_nombre(self):
        nombres = defaultdict(int)
        self._calcular_suma_cantidad_por_nombre_recursivo(self.raiz, nombres)
        return dict(nombres)

    def _calcular_suma_cantidad_por_nombre_recursivo(self, nodo, nombres):
        if nodo.producto:
            producto_actual = nodo.producto
            while producto_actual:
                nombres[producto_actual.nombre] += producto_actual.cantidad
                producto_actual = producto_actual.siguiente
        if nodo.hijo:
            self._calcular_suma_cantidad_por_nombre_recursivo(nodo.hijo, nombres)
        if nodo.hermano:
            self._calcular_suma_cantidad_por_nombre_recursivo(nodo.hermano, nombres)

    def calcular_suma_cantidad_por_categoria(self):
        categorias = defaultdict(int)
        self._calcular_suma_cantidad_por_categoria_recursivo(self.raiz, categorias)
        return dict(categorias)

    def _calcular_suma_cantidad_por_categoria_recursivo(self, nodo, categorias):
        if nodo.producto:
            producto_actual = nodo.producto
            while producto_actual:
                categorias[producto_actual.categoria] += producto_actual.cantidad
                producto_actual = producto_actual.siguiente
        if nodo.hijo:
            self._calcular_suma_cantidad_por_categoria_recursivo(nodo.hijo, categorias)
        if nodo.hermano:
            self._calcular_suma_cantidad_por_categoria_recursivo(nodo.hermano, categorias)

# Función para cargar datos desde archivo
def cargar_datos_desde_archivo(nombre_archivo, arbol):
    with open(nombre_archivo, 'r') as archivo:
        lineas = archivo.readlines()
        for linea in lineas:
            datos = json.loads(linea.strip())
            tipo = "Perecedero" if datos["tipo"] == "perecedero" else "No perecedero"
            producto = Producto(
                nombre=datos["nombre"],
                fecha_vencimiento=datos.get("fech_venc"),
                perecible=(datos["tipo"] == "perecedero"),
                cantidad=int(datos["cantidad"]),  # Convertir a entero
                categoria=datos["categoria"]
            )
            # Buscar o crear el nodo del tipo correspondiente
            nodo_tipo = arbol.raiz.buscar_producto_por_tipo(tipo)
            if not nodo_tipo:
                nodo_tipo = Nodo(tipo)
                arbol.agregar_tipo(nodo_tipo)
            # Agregar el producto al nodo
            nodo_tipo.agregar_producto(producto)

# Crear el árbol de productos
arbol_productos = Arbol("Almacén")

# Cargar datos desde inventario.txt
cargar_datos_desde_archivo("inventario.txt", arbol_productos)

In [2]:
# Mostrar los productos perecederos y no perecederos
perecederos = arbol_productos.buscar_por_tipo("Perecedero")
no_perecederos = arbol_productos.buscar_por_tipo("No perecedero")

print("Productos perecederos:")
while perecederos:
    print(f"- {perecederos.nombre}, vence: {perecederos.fecha_vencimiento.strftime('%Y-%m-%d')}")
    perecederos = perecederos.siguiente

print("\nProductos no perecederos:")
while no_perecederos:
    print(f"- {no_perecederos.nombre}")
    no_perecederos = no_perecederos.siguiente

# Buscar productos perecederos que vencen antes de una fecha
print("\nProductos perecederos que vencen antes de 2024-11-05:")
arbol_productos.buscar_por_vencimiento("2024-11-05")

Productos perecederos:
- Manzana, vence: 2024-10-26
- Pollo, vence: 2024-10-27
- Naranja, vence: 2024-11-15
- Platano, vence: 2025-01-12
- Leche, vence: 2025-11-10

Productos no perecederos:
- Arroz
- Lentejas
- Pasta

Productos perecederos que vencen antes de 2024-11-05:
Producto que vence antes del 2024-11-05: Manzana
Producto que vence antes del 2024-11-05: Pollo


In [3]:
# Cálculo de sumas
print("Cantidad de productos por tipo:")
suma_por_tipo = arbol_productos.calcular_suma_cantidad_por_tipo()
print(", ".join([f"{tipo}: {cantidad}" for tipo, cantidad in suma_por_tipo.items()]))

print("\nCantidad de productos por nombre:")
suma_por_nombre = arbol_productos.calcular_suma_cantidad_por_nombre()
print(", ".join([f"{nombre}: {cantidad}" for nombre, cantidad in suma_por_nombre.items()]))

print("\nCantidad de productos por categoría:")
suma_por_categoria = arbol_productos.calcular_suma_cantidad_por_categoria()
print(", ".join([f"{categoria}: {cantidad}" for categoria, cantidad in suma_por_categoria.items()]))

Cantidad de productos por tipo:
Perecedero: 157, No perecedero: 240

Cantidad de productos por nombre:
Manzana: 50, Pollo: 20, Naranja: 2, Platano: 55, Leche: 30, Arroz: 100, Lentejas: 80, Pasta: 60

Cantidad de productos por categoría:
fruta: 107, carnes: 20, lacteo: 30, grano: 180, pasta: 60
